In [14]:
'''
This note book is a simple implementation of the Heston model for the price of a European call option.

Good references:
https://hal.sorbonne-universite.fr/hal-02273889/document
https://www.maths.univ-evry.fr/pages_perso/crepey/Finance/051111_mikh%20heston.pdf
https://quantpy.com.au/stochastic-volatility-models/heston-model-calibration-to-option-prices/
https://calebmigosi.medium.com/build-the-heston-model-from-scratch-part-i-17bde00122a0 (series of 3 articles)
https://github.com/CalebMigosi/code-more/tree/main/EquityOptionsPricing

Some expressions vary between the sources. Formulas are mainly based on Liang and Cai (2022) and Pironneau (2019). 
'''

'\nThis note book is a simple implementation of the Heston model for the price of a European call option.\n\nTwo good references are:\nhttps://calebmigosi.medium.com/build-the-heston-model-from-scratch-part-i-17bde00122a0 (series of 3 articles)\nhttps://quantpy.com.au/stochastic-volatility-models/heston-model-calibration-to-option-prices/\n\nThis implement is mostly based on the first reference, where the github repo is available at:\nhttps://github.com/CalebMigosi/code-more/tree/main/EquityOptionsPricing\n'

In [1]:
import numpy as np
import pandas as pd
from numba import njit, jit, prange #For parallelization to accelerate run time
from lmfit import Parameters, minimize
from scipy.integrate import quad
import cProfile # Analyze run time
from multiprocessing import Pool, cpu_count
from functools import partial

import time

i = complex (0,1) # Define complex number i

In [2]:
file = '../data/processed_data/2020_2022_moneyness_filtere.csv'
df_read = pd.read_csv(file)
df_read['TTM'] = df_read['TTM'] / 365
df_read['R'] = df_read['R'] / 100

@jit
def create_data(df, date):
    df = df[df['Quote_date'] == date]
    optionPrices = df['Price'].to_numpy('float')
    S0 = df['Underlying_last'].to_numpy('float')[0]
    strikes = df['Strike'].to_numpy('float')
    rates = df['R'].to_numpy('float')
    maturities = df['TTM'].to_numpy('float')
    return df, optionPrices, S0, strikes, rates, maturities

In [3]:
@jit
def charHeston(u, S0, r, T, sigma, kappa, theta, v0, rho):
    '''Implementation of the characteristic function of the Heston model'''
    # Frequent expression
    rsiu = rho*sigma*i*u

    # Calculate d
    d1 = (rsiu - kappa)**2
    d2 = sigma**2*(i*u + u**2)
    d = np.sqrt(d1 + d2)

    # Calculate g
    g1 = kappa - rsiu - d
    g2 = kappa - rsiu + d
    g = g1/g2

    #Calculate first exp
    exp1 = np.exp(r*T)

    #Calculate the first power
    base1 = S0
    exponent1 = i*u
    power1 = np.power(base1, exponent1)

    # Calculate second power
    base2 = (1-g*np.exp(-d*T)) / (1-g)
    exponent2 = -2*theta*kappa/(sigma**2)
    power2 = np.power(base2, exponent2)

    # Calculate the second exp
    part1 = theta*kappa*T/(sigma**2) * g1
    part2 = v0/(sigma**2) * g2 * (1 - np.exp(d*T))/(1 - g*np.exp(d*T))
    exp2 = np.exp(part1 + part2)

    # Main calculation
    return exp1*power1*power2*exp2

@jit
def integrand(u, S0, K, r, T, sigma, kappa, theta, v0, rho):
    '''Calculate the integrand of the Heston model'''
    numerator = np.exp(r*T)*charHeston(u-i, S0, r, T, sigma, kappa, theta, v0, rho) - K * charHeston(u, S0, r, T, sigma, kappa, theta, v0, rho)
    denominator = i*u *np.power(K, i*u)
    return np.real(numerator/denominator)

@jit(forceobj = True)
def priceHestonIntegral(S0, K, r, T, sigma, kappa, theta, v0, rho, maxIntegral = 100):
    '''Calculate integral for the price of a European call option using the Heston model'''
    integral = np.array([quad(integrand, 0, maxIntegral, args=(S0, K_i, r_i, T_i, sigma, kappa, theta, v0, rho))[0] for K_i, r_i, T_i in zip(K, r, T) ])
    return 0.5 * (S0 - K * np.exp(-r * T)) + integral/np.pi

def iter_cb(params, iter, resid):
    '''Callback function to print the parameters at each iteration of the minimizer'''
    parameters = [params['sigma'].value, 
                  params['kappa'].value, 
                  params['theta'].value, 
                  params['v0'].value, 
                  params['rho'].value, 
                  np.sum(resid)/len(resid)]
    print(parameters) 

def calibrateHeston(optionPrices, S0, strikes, rates, maturities):
    '''Calibrate the Heston model parameters using the Levenberg Marquardt algorithm'''

    # Define the parameters to calibrate
    params = Parameters()
    params.add('sigma',value = 0.028, min = 1e-3, max = 1)
    params.add('kappa',value = 0.042, min = 1e-3, max = 5)
    params.add('theta',value = 0.001, min = 1e-4, max = 0.5)
    params.add('v0', value = 0.028, min = 1e-3, max = 0.5)
    params.add('rho', value = -6e-7, min = -1, max = 0)
    
    # Define the objective function to minimize as squared errors
    objectiveFunctionHeston = lambda paramVect: (optionPrices - priceHestonIntegral(S0, strikes,  
                                                                        rates, 
                                                                        maturities, 
                                                                        paramVect['sigma'].value,                         
                                                                        paramVect['kappa'].value,
                                                                        paramVect['theta'].value,
                                                                        paramVect['v0'].value,
                                                                        paramVect['rho'].value)) **2   
    # Run the Levenberg Marquardt algorithm
    result = minimize(objectiveFunctionHeston, 
                      params, 
                      method = 'leastsq',
#                      iter_cb = iter_cb,
                      ftol = 1e-3) 
    return(result)


In [4]:
cal_date = '2022-12-29'
test_date = '2021-01-05'
optionPrices, S0, strikes, rates, maturities = create_data(df_read, cal_date)
optionPrices_test, S0_test, strikes_test, rates_test, maturities_test = create_data(df_read, test_date)

print(f'Size of calibration set: {len(optionPrices)}')
print(f'Size of test set: {len(optionPrices_test)}')

result = calibrateHeston(optionPrices, S0, strikes, rates, maturities, initialParameters=[0.15946199040726322, 0.5270090754510403, 0.017571026540824892, 0.010000000000003327, -0.23523355194337947])

Size of calibration set: 6799
Size of test set: 6508
[0.15946199040726322, 0.5270090754510403, 0.017571026540824892, 0.010000000000003327, -0.23523355194337947, 1.4730965029269536e+24]
[0.15946199040726322, 0.5270090754510403, 0.017571026540824892, 0.010000000000003327, -0.23523355194337947, 1.4730965029269536e+24]
[0.15946199040726322, 0.5270090754510403, 0.017571026540824892, 0.010000000000003327, -0.23523355194337947, 1.4730965029269536e+24]
[0.1594620143619902, 0.5270090754510403, 0.017571026540824892, 0.010000000000003327, -0.23523355194337947, 1.4730978256015151e+24]
[0.15946199040726322, 0.5270091121147864, 0.017571026540824892, 0.010000000000003327, -0.23523355194337947, 1.473095892842378e+24]
[0.15946199040726322, 0.5270090754510403, 0.017571032750055383, 0.010000000000003327, -0.23523355194337947, 1.473096293326786e+24]
[0.15946199040726322, 0.5270090754510403, 0.017571026540824892, 0.010000000000008873, -0.23523355194337947, 1.473096502924777e+24]
[0.15946199040726322, 0.527

In [87]:
print(result.params)
print(result.params['sigma'].value)

Parameters([('sigma', <Parameter 'sigma', value=0.15946199040726322 +/- 0.000994, bounds=[0.01:10]>), ('kappa', <Parameter 'kappa', value=0.5270090754510403 +/- 0.00156, bounds=[0.01:10]>), ('theta', <Parameter 'theta', value=0.017571026540824892 +/- 0.000504, bounds=[0.01:10]>), ('v0', <Parameter 'v0', value=0.010000000000003327 +/- 9.49e-05, bounds=[0.01:10]>), ('rho', <Parameter 'rho', value=-0.23523355194337947 +/- 0.00218, bounds=[-1:0]>)])
0.15946199040726322


In [15]:
initial_params = [0.03, 0.04, 0.01, 0.03, -0.002]
_, optionPrices_cal, S0_cal, strikes_cal, rates_cal, maturities_cal = create_data(df_read, '2022-12-28')
calibrationResult = calibrateHeston(optionPrices_cal, S0_cal, strikes_cal, rates_cal, maturities_cal, initialParameters=initial_params)

[0.03000000000000002, 0.039999999999999876, 0.01000000000000002, 0.030000000000000013, -0.0020000000000000018, 145.0691443035268]
[0.03000000000000002, 0.039999999999999876, 0.01000000000000002, 0.030000000000000013, -0.0020000000000000018, 145.0691443035268]
[0.03000000000000002, 0.039999999999999876, 0.01000000000000002, 0.030000000000000013, -0.0020000000000000018, 145.0691443035268]
[0.030000006784357977, 0.039999999999999876, 0.01000000000000002, 0.030000000000000013, -0.0020000000000000018, 145.06924163136776]
[0.03000000000000002, 0.040000011901532774, 0.01000000000000002, 0.030000000000000013, -0.0020000000000000018, 145.06901697819004]
[0.03000000000000002, 0.039999999999999876, 0.01000000194201045, 0.030000000000000013, -0.0020000000000000018, 145.06914459614026]
[0.03000000000000002, 0.039999999999999876, 0.01000000000000002, 0.030000003069964466, -0.0020000000000000018, 145.06913912095305]
[0.03000000000000002, 0.039999999999999876, 0.01000000000000002, 0.030000000000000013

In [4]:
@jit
def rmse(predictions, targets):
    return (np.sum((predictions - targets) ** 2) / len(predictions))**(0.5)

@jit(parallel=True, forceobj=True)
def priceHestonPeriod(df):
    '''Calibrate the Heston model parameters and price options across a time period'''
    dates = np.sort(df['Quote_date'].unique())
    print(dates)

    initial_params = np.array([0.03, 0.04, 0.01, 0.03, -0.002])
    params = np.empty((len(dates) - 1, 5))
    optionPricesHeston = np.empty((len(df),))

    for i in prange(len(dates) - 1):
        # Create the calibration and test sets
        _, optionPrices_cal, S0_cal, strikes_cal, rates_cal, maturities_cal = create_data(df, dates[i])
        df_test, optionPrices_test, S0_test, strikes_test, rates_test, maturities_test = create_data(df, dates[i+1])

        # Calibrate the Heston model
        calibrationResult = calibrateHeston(optionPrices_cal, S0_cal, strikes_cal, rates_cal, maturities_cal, initialParameters=initial_params)
        params[i, :] = np.array([calibrationResult.params['sigma'].value, calibrationResult.params['kappa'].value, calibrationResult.params['theta'].value, calibrationResult.params['v0'].value, calibrationResult.params['rho'].value])
        print(f'Calibration for date: {dates[i]} RMSE {(np.sum(calibrationResult.residual)/len(optionPrices_cal))**(0.5)} and parameters {params[i, :]}')

        # Price the options
        optionPricesHeston[df['Quote_date'] == dates[i+1]] = priceHestonIntegral(S0_test, strikes_test, rates_test, maturities_test, *params[i, :])
        print(f'Test date {dates[i+1]}: RMSE: {rmse(optionPricesHeston[df["Quote_date"] == dates[i+1]], optionPrices_test)}')

    df['Heston_price'] = optionPricesHeston
    df = df[df['Quote_date'] != dates[0]]
    print('=====================')
    print(f'Total RMSE: {rmse(df["Heston_price"], df["Price"])}')
    return df

@jit(forceobj=True)
def create_data_np_grouped(group):
    '''Create numpy arrays with required data for calibration and testing'''
    optionPrices = group['Price'].values
    S0 = group['Underlying_last'].values[0]
    strikes = group['Strike'].values
    rates = group['R'].values
    maturities = group['TTM'].values

    data_np = np.empty((len(optionPrices), 5))
    data_np[:, 0] = optionPrices
    data_np[:, 1] = S0
    data_np[:, 2] = strikes
    data_np[:, 3] = rates
    data_np[:, 4] = maturities

    return data_np


@jit(parallel=True, forceobj=True)
def priceHestonPeriod_np(df):
    '''Calibrate the Heston model parameters and price options across a time period'''
    dates = np.sort(df['Quote_date'].unique())
    print(dates)

    # Group the data by date and apply create_data_np_grouped to each group
    data_nps = df.groupby('Quote_date').apply(create_data_np_grouped)

    initial_params = np.array([0.03, 0.04, 0.01, 0.03, -0.002])
    params = np.empty((len(dates) - 1, 5))
    optionPricesHeston = np.empty((len(df) - len(data_nps[0][:,0]),))

    index = 0 

    # Calibrate and price options for each date
    for i in prange(len(dates)-1):
        data_cal = data_nps[i]
        data_test = data_nps[i+1]

        # Calibrate the Heston model
        calibrationResult = calibrateHeston(
            data_cal[:, 0],
            data_cal[0, 1],
            data_cal[:, 2],
            data_cal[:, 3],
            data_cal[:, 4],
            initialParameters=initial_params
        )

        params[i, :] = np.array([
            calibrationResult.params['sigma'].value,
            calibrationResult.params['kappa'].value,
            calibrationResult.params['theta'].value,
            calibrationResult.params['v0'].value,
            calibrationResult.params['rho'].value
        ])

        print(f'Calibration for date: {dates[i]} RMSE {(np.sum(calibrationResult.residual)/len(data_cal[:, 0]))**(0.5)} and parameters {params[i, :]}')

        # Price the options
        end_index = index + len(data_test[:,0])
        optionPricesHeston[index : end_index ] = priceHestonIntegral(
            data_test[0, 1],
            data_test[:, 2],
            data_test[:, 3],
            data_test[:, 4],
            *params[i, :]
        )
        print(f'Test date {dates[i+1]}: RMSE: {(np.sum((optionPricesHeston[index : end_index] - data_test[:,0]) ** 2) / len(data_test[:,0]))**(0.5)}')
        index = end_index

    df = df[df['Quote_date'] != dates[0]]
    df['Heston_price'] = optionPricesHeston
    print('=====================')
    print(f'Total RMSE: {(np.sum((df["Heston_price"] - df["Price"]) ** 2) / len(df["Price"]))**(0.5)}')
    return df

In [21]:
df_testing = df_read[df_read['Quote_date'] >= '2022-12-29']
t = time.time()
priceHestonPeriod(df_testing)
print(f'Time regular first setup: {time.time() - t}')
df_testing = df_read[df_read['Quote_date'] >= '2022-12-28']
t = time.time()
priceHestonPeriod(df_testing)
print(f'Time regular 2 runs setup: {time.time() - t}')

['2022-12-29' '2022-12-30']
Calibration for date: 2022-12-29 RMSE 11.112025053860684 and parameters [ 2.83463075e-02  4.21754748e-02  1.16421382e-03  2.74010313e-02
 -5.53674773e-10]
Test date 2022-12-30: RMSE: 11.227801919582385
Total RMSE: 11.227801919582383
Time regular first setup: 142.96203994750977


C:\Users\Erlend\AppData\Local\Temp\ipykernel_3824\344846862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priceHestonPeriod(df_testing)
C:\Users\Erlend\AppData\Local\Temp\ipykernel_3824\3115993926.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "rmse" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\Erlend\AppData\Local\Temp\ipykernel_3824\3115993926.py (3)

File "..\..\..\..\..\..\..\AppData\Local\Temp\ipykernel_3824\3115993926.py", line 3:
<source missing, REPL/exec in use?>

  @jit
c:\Users\Erlend\AppData\Local\Programs\Python\Python39\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "rmse" was compiled in

['2022-12-28' '2022-12-29' '2022-12-30']
Calibration for date: 2022-12-28 RMSE 11.297207601581327 and parameters [ 2.86063610e-02  4.18566659e-02  1.00025391e-03  2.89287021e-02
 -6.14412305e-07]
Test date 2022-12-29: RMSE: 10.852432229481208
Calibration for date: 2022-12-29 RMSE 11.112025053860684 and parameters [ 2.83463075e-02  4.21754748e-02  1.16421382e-03  2.74010313e-02
 -5.53674773e-10]
Test date 2022-12-30: RMSE: 11.227801919582385
Total RMSE: 11.038024951998926
Time regular 2 runs setup: 233.37559509277344


C:\Users\Erlend\AppData\Local\Temp\ipykernel_3824\344846862.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priceHestonPeriod(df_testing)


In [22]:
df_testing = df_read[df_read['Quote_date'] >= '2022-12-29']
t = time.time()
priceHestonPeriod_np(df_testing)
print(f'Time np first setup: {time.time() - t}')
df_testing = df_read[df_read['Quote_date'] >= '2022-12-28']
t = time.time()
priceHestonPeriod_np(df_testing)
print(f'Time np 2 runs setup: {time.time() - t}')

['2022-12-29' '2022-12-30']
Calibration for date: 2022-12-29 RMSE 11.112025053860684 and parameters [ 2.83463075e-02  4.21754748e-02  1.16421382e-03  2.74010313e-02
 -5.53674773e-10]
Test date 2022-12-30: RMSE: 11.227801919582383
Total RMSE: 11.227801919582383
Time np first setup: 139.00138306617737


C:\Users\Erlend\AppData\Local\Temp\ipykernel_3824\4261483577.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priceHestonPeriod_np(df_testing)


['2022-12-28' '2022-12-29' '2022-12-30']
Calibration for date: 2022-12-28 RMSE 11.297207601581327 and parameters [ 2.86063610e-02  4.18566659e-02  1.00025391e-03  2.89287021e-02
 -6.14412305e-07]
Test date 2022-12-29: RMSE: 10.852432229481215
Calibration for date: 2022-12-29 RMSE 11.112025053860684 and parameters [ 2.83463075e-02  4.21754748e-02  1.16421382e-03  2.74010313e-02
 -5.53674773e-10]
Test date 2022-12-30: RMSE: 11.227801919582383
Total RMSE: 11.038024951998926
Time np 2 runs setup: 229.8867678642273


C:\Users\Erlend\AppData\Local\Temp\ipykernel_3824\4261483577.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priceHestonPeriod_np(df_testing)


In [6]:
if __name__ == '__main__':
    df = df_read[df_read['Quote_date'] >= '2022-12-29']
    t = time.time()
    dates = np.sort(df['Quote_date'].unique())
    print(dates)

    # Group the data by date and apply create_data_np_grouped to each group
    data_nps = df.groupby('Quote_date').apply(create_data_np_grouped)
    data_nps = [(data_cal[:, 0],
        data_cal[0, 1],
        data_cal[:, 2],
        data_cal[:, 3],
        data_cal[:, 4]) for data_cal in data_nps[:-1]]

    with Pool(cpu_count()) as p:
        params = p.starmap(calibrateHeston, data_nps)
        print(params)

    print(f'Time mp 1 runs setup: {time.time() - t}')

    df = df_read[df_read['Quote_date'] >= '2022-12-28']
    t = time.time()
    dates = np.sort(df['Quote_date'].unique())
    print(dates)

    # Group the data by date and apply create_data_np_grouped to each group
    data_nps = df.groupby('Quote_date').apply(create_data_np_grouped)
    data_nps = [(data_cal[:, 0],
        data_cal[0, 1],
        data_cal[:, 2],
        data_cal[:, 3],
        data_cal[:, 4]) for data_cal in data_nps[:-1]]

    with Pool(cpu_count()) as p:
        params = p.starmap(calibrateHeston, data_nps)
        print(params)
    print(f'Time mp 2 runs setup: {time.time() - t}')

['2022-12-29' '2022-12-30']


In [18]:
def price_options(market_prices, St, K, r, T, sigma, kappa, theta, v0, rho):
    prices = priceHestonIntegral(St, K, r, T, sigma, kappa, theta, v0, rho)
    return np.sum((market_prices - prices)**2)/len(prices)

params = [0.02286668553347334, 0.03415432023577888, 0.010000000021317148, 0.027453577229969835, -9.83324943248931e-07]

err = price_options(*create_data(df_read, '2021-01-04'), *params)
print(err, err**(1/2))
err = price_options(*create_data(df_read, '2021-01-05'), *params)
print(err, err**(1/2))
err = price_options(*create_data(df_read, '2022-12-29'), *params)
print(err, err**(1/2))
err = price_options(*create_data(df_read, '2022-12-30'), *params)
print(err, err**(1/2))



1006.9845909005606 31.73302051334793
912.3592073211577 30.205284427085896
122.53155352522978 11.069397161780302
124.98141417674441 11.179508673315855


In [5]:
#Check if integral and rectangle yields the same result
marketPrices = df_test['Price'].to_numpy('float')[:5]
strikes = df_test['Strike'].to_numpy('float')[:5]
rates = df_test['R'].to_numpy('float')[:5]
maturities = df_test['TTM'].to_numpy('float')[:5]

sigma, kappa, theta, v0, rho = 0.15946199, 0.52700908, 0.01757103, 0.01000000, -0.23523355
S0 = 3727.05
print(priceHestonIntegral(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho))
print(priceHeston_rec(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho))
print(marketPrices)

[2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
[2029.19410008 1929.34020049 1825.11753565 1717.68419354 1618.36493904]
[2026.7   1926.705 1826.65  1726.3   1626.395]


In [12]:
for i in range(50):
    time1 = time.time()
    print(f'i = {i}, {priceHestonIntegral(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho)}')
    print(f'Time: {time.time() - time1}')

i = 0, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.0055391788482666016
i = 1, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.003036975860595703
i = 2, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.002704620361328125
i = 3, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.001001596450805664
i = 4, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.001992464065551758
i = 5, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.002903461456298828
i = 6, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.0016312599182128906
i = 7, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.003997087478637695
i = 8, [2034.25819887 1933.80400267 1829.03539769 1721.1057716  1621.33582195]
Time: 0.004557371139526367
i = 9, [2034.25819887 1933.80400267 1829.035

In [11]:
for i in range(50):
    time1 = time.time()
    priceHeston_rec(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho)
    print(f'Time: {time.time() - time1}')


C:\Users\Erlend\AppData\Local\Temp\ipykernel_34880\1317513057.py:3: RuntimeWarning: divide by zero encountered in true_divide
  print(f'i = {i}, {priceHeston_rec(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho)}')
C:\Users\Erlend\AppData\Local\Temp\ipykernel_34880\1317513057.py:3: RuntimeWarning: invalid value encountered in add
  print(f'i = {i}, {priceHeston_rec(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho)}')


i = 0, [nan nan nan nan nan]
Time: 0.32698869705200195


C:\Users\Erlend\AppData\Local\Temp\ipykernel_34880\1317513057.py:3: RuntimeWarning: overflow encountered in multiply
  print(f'i = {i}, {priceHeston_rec(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho)}')
C:\Users\Erlend\AppData\Local\Temp\ipykernel_34880\1317513057.py:3: RuntimeWarning: overflow encountered in power
  print(f'i = {i}, {priceHeston_rec(S0, strikes, rates, maturities, sigma, kappa, theta, v0, rho)}')


i = 1, [ 115.7127265    14.56261903  -86.49603888 -187.46810312 -288.35794461]
Time: 0.28809022903442383
i = 2, [660.93585763 591.58444695 522.30081736 453.08108553 383.92177358]
Time: 0.3336334228515625
i = 3, [826.98153109 766.99537719 707.05581027 647.16008325 587.30574091]
Time: 0.3028569221496582
i = 4, [899.95953015 844.01078122 788.09545899 732.21156097 676.35729928]
Time: 0.2773261070251465
i = 5, [938.47438631 884.62243503 830.79543819 776.99188416 723.21042424]
Time: 0.2789731025695801
i = 6, [961.28055282 908.64581986 856.03033836 803.43292859 750.85253882]
Time: 0.27809572219848633
i = 7, [975.7229988  923.8505555  871.99329317 820.15027197 768.32065459]
Time: 0.28908371925354004
i = 8, [985.35952662 933.99094805 882.63452762 831.28950632 779.95520822]
Time: 0.2911081314086914
i = 9, [992.111717   941.08926129 890.07669586 839.07339709 788.07881001]
Time: 0.2837700843811035
i = 10, [996.96287103 946.18594529 895.41715201 844.65597328 793.90194854]
Time: 0.2869453430175781
i

KeyboardInterrupt: 

In [ ]:
@jit(forceobj = True)
def priceHeston_rec(S0, K, r, T, sigma, kappa, theta, v0, rho, max = 100):
    P, iter = 0, 10000
    ds = max/iter

    for j in prange(1, iter):
        u = ds * (2*j + 1)/2
        numerator = np.exp(r*T) * charHeston(u-i, S0, r, T, sigma, kappa, theta, v0, rho) - K*charHeston(u, S0, r, T, sigma, kappa, theta, v0, rho)
        denominator = i*u*np.power(K, i*u)
        P += ds * numerator/denominator
    return 0.5*(S0 - K*np.exp(-r*T)) + np.real(P)/np.pi


# This is the calibration function
def calibratorHeston_rec(St, initialValues = [0.1629707464827465, 0.3814448150689377, 0.010031155605291166, 0.042860179235966044, -0.023904162408956653], 
                              lowerBounds = [1e-2,1e-2,1e-2,1e-2,-1], 
                              upperBounds = [10,10,10,10,0]):
    params = Parameters()
    params.add('sigma',value = initialValues[0], min = lowerBounds[0], max = upperBounds[0])
    params.add('kappa',value = initialValues[1], min = lowerBounds[1], max = upperBounds[1])
    params.add('theta',value = initialValues[2], min = lowerBounds[2], max = upperBounds[2])
    params.add('v0', value = initialValues[3], min = lowerBounds[3], max = upperBounds[3])
    params.add('rho', value = initialValues[4], min = lowerBounds[4], max = upperBounds[4])
    
    
    objectiveFunctionHeston = lambda paramVect: (marketPrices - priceHeston_rec(St, strikes,  
                                                                        rates, 
                                                                        maturities, 
                                                                        paramVect['sigma'].value,                         
                                                                        paramVect['kappa'].value,
                                                                        paramVect['theta'].value,
                                                                        paramVect['v0'].value,
                                                                        paramVect['rho'].value)) **2   
    result = minimize(objectiveFunctionHeston, 
                      params, 
                      method = 'leastsq',
                      iter_cb = iter_cb,
                      ftol = 1e-6,
                      max_nfev = 3)
    return(result)

In [ ]:
calibratorHeston_(3701.38)

[0.1630997308063169, 0.38104045661636404, 0.010160402051625007, 0.042969837388819265, -0.022018903191991956, 4212.973655313243]
[0.1630997308063169, 0.38104045661636404, 0.010160402051625007, 0.042969837388819265, -0.022018903191991956, 4212.973655313243]
[0.1630997308063169, 0.38104045661636404, 0.010160402051625007, 0.042969837388819265, -0.022018903191991956, 4212.973655313243]
[0.16309975499179813, 0.38104045661636404, 0.010160402051625007, 0.042969837388819265, -0.022018903191991956, 4212.974483298296]
[0.1630997308063169, 0.3810404899170622, 0.010160402051625007, 0.042969837388819265, -0.022018903191991956, 4212.97305948086]
[0.1630997308063169, 0.38104045661636404, 0.010160402983812688, 0.042969837388819265, -0.022018903191991956, 4212.973665303213]
[0.1630997308063169, 0.38104045661636404, 0.010160402051625007, 0.04296984981839134, -0.022018903191991956, 4212.974511199293]
[0.1630997308063169, 0.38104045661636404, 0.010160402051625007, 0.042969837388819265, -0.02201890040853449

KeyboardInterrupt: 